# Preparing CRSP data for backtesting (monthly data)

&copy; **Johannes Ruf** (comments welcome under j.ruf@lse.ac.uk, February 2023)

In this notebook, we construct a dataframe `df` that can be used to backtest systematic trading strategies. The class of trading strategies to be considered are strategies that are functions of the stock capitalizations only (and don't depend on other characteristics, e.g., industries). 

The dataframe `df` will have three components: a matrix of returns, a matrix of market capitalizations, and a matrix of flags that tag problematic returns.

A flag value of `0` implies no special issues. The remaining flag values are constructed as follows.

In [1]:
FLAG_PROBLEMATIC_INTERMEDIATE_RETURN = 2

FLAG_TEMPORARY_DELISTING = 3

FLAG_DELRET_MISSING = 4

FLAG_RETURN_BASED_ON_BA = 1    # if return is based on a bid-ask average and none of the above three flags valid

Moreover, we add 10 to the flag value if the corresponding return is larger/smaller than the following cutoff:

In [2]:
CUTOFF_LARGE_RETURN = 1   # 1 corresponds to doubling over the month, i.e., a period return of 100%.
CUTOFF_SMALL_RETURN = -0.5

If securities are 'temporary delisted' (here defined as previous capitalization is available but return is missing, within a time series), we replace the first missing return in a consecutive sequence of missing returns by `TEMPORARY_DELISTING_RETURN`.

If delisting returns are missing, we set them to `MISSING_DELIST_RETURN`. 

Of course, other values or methods to handle these returns are possible, too.  When backtesting trading strategies, robustness checks with respect to these assumptions are recommended (easily implemented thanks to the backtesting flags constructed below).

In [3]:
TEMPORARY_DELISTING_RETURN = -0.1
MISSING_DELIST_RETURN = -0.3

## Preparations

In [4]:
import pandas as pd

import wrds
WRDS_LOGIN = 'pedrou'    # update to your login info on CRSP

DATAPATH = '~/Downloads/mathematical_finance/data/'

In [5]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


## Loading the data

The trading strategies we will consider may depend on the stocks' capitalizations. To avoid 'anticipatory' strategies, at any month we are only allowed to use the previous months' capitalizations.

In [6]:
%%time

df = db.raw_sql("""SELECT mthcaldt, permno, mthprevcap, mthret,
                          mthprcflg, mthdelflg, mthretflg 
                          FROM crsp.StkMthSecurityData
                          WHERE 
                          sharetype = 'NS' AND securitysubtype = 'COM' 
                          AND issuertype IN ('ACOR','CORP') AND usincflg = 'Y'
                          """, date_cols='mthcaldt')

CPU times: user 18.3 s, sys: 2 s, total: 20.3 s
Wall time: 46.9 s


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3791141 entries, 0 to 291140
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   mthcaldt    datetime64[ns]
 1   permno      float64       
 2   mthprevcap  float64       
 3   mthret      float64       
 4   mthprcflg   object        
 5   mthdelflg   object        
 6   mthretflg   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 231.4+ MB


## Outline of the cleaning steps

We now proceed with the following steps:
1) First, we do some preliminary cleaning steps and add a column to flag critical returns.
2) We pivot the data so that each row corresponds to one date, and each column to a `permno`.
3) We check and clean the beginning and end of each time series.
4) We check and clean for 'temporary delistings'.
5) We flag very large/small returns.
6) We store the data.

## Preliminary cleaning steps

Here, we check whether market capitalizations are available, whether returns are based on bid-ask spreads, and whether CRSP tagged returns as problematic (in particular, if intermediate returns are missing).

#### Market capitalizations

If we don't observe a market capitalization, we assume that we don't trade that security in the following period. Hence, we remove it from the investment universe:

In [8]:
bl = df['mthprevcap'].isnull()

In [9]:
print('This step removes {:_} ({:.2f}%) rows.'.format(bl.sum(), 100 * bl.sum() / len(df)))

This step removes 75_204 (1.98%) rows.


In [10]:
df = df[~bl]

#### A new column and prices based on bid-ask-spreads

We now add a new column called `bcktstflg` ('backtesting flag') to the dataframe, where we flag all problematic returns. 

We first flag all returns that are based on a bid-ask-average instead of a trading price. (Those returns might be re-tagged if they correspond to a problematic intermediate return or a missing delisting return.)

In [11]:
bl = df['mthprcflg'].eq('BA')
df = df.drop('mthprcflg', axis=1)

In [12]:
print('This step tags {:_} ({:.2f}%) rows.'.format(bl.sum(), 100 * bl.sum() / len(df)))

This step tags 727_523 (19.58%) rows.


In [13]:
df['bcktstflg'] = 0
df.loc[bl, 'bcktstflg'] = FLAG_RETURN_BASED_ON_BA

#### Intermediate missing and problematic returns

##### Preamble

Compared to the legacy data, the new flat file format 2.0 has less missing price and return information. Two examples for this are `permno` 10346 (in 1996) and `permno` 10346 (in 1994).

In [14]:
db.raw_sql("SELECT mthcaldt, mthprc, mthret, mthcompflg, mthretflg FROM crsp.StkMthSecurityData WHERE permno = 10155", 
           date_cols='mthcaldt').set_index('mthcaldt').loc['1996-08']

,mthprc,mthret,mthcompflg,mthretflg
mthcaldt,,,,
1996-08-30,0.875,-0.222222,R,GP


In [15]:
db.raw_sql("SELECT date, prc, ret FROM crsp.msf WHERE permno = 10155", 
           date_cols='date').set_index('date').loc['1996-8']

,prc,ret
date,,
1996-08-30,0.875,NaN


In [16]:
db.raw_sql("SELECT mthcaldt, mthprc, mthret, mthcompflg, mthretflg FROM crsp.StkMthSecurityData WHERE permno = 10346", 
           date_cols='mthcaldt').set_index('mthcaldt').loc['1994-05':'1994-09']

,mthprc,mthret,mthcompflg,mthretflg
mthcaldt,,,,
1994-05-31,0.015625,-5.000000e-01,R,IP
1994-06-30,0.046875,2.000000e+00,R,IP
1994-07-29,0.015625,-6.666667e-01,R,IP
1994-08-31,0.015625,4.440892e-16,R,IP
1994-09-30,NaN,NaN,R,NT


In [17]:
db.raw_sql("SELECT date, prc, ret FROM crsp.msf WHERE permno = 10346", 
           date_cols='date').set_index('date').loc['1994-05':'1994-09']

,prc,ret
date,,
1994-05-31,NaN,NaN
1994-06-30,0.046875,0.5
1994-07-29,NaN,NaN
1994-08-31,NaN,NaN
1994-09-30,NaN,NaN


However, we have to be very careful with these returns. In the new data format, problematic prices and returns in the monthly file are flagged via the `mthcompflg` [flag](https://www.crsp.org/files/appendix/FlagType_CF.html) and the `mthretflg` [flag](https://www.crsp.org/files/appendix/FlagType_AR.html).

##### Marking problematic and missing intermediate returns

In [18]:
bl = df['mthretflg'].isin(['NT', 'MP', 'IP', 'GP'])
df = df.drop('mthretflg', axis=1)

The above code captures all problematic returns. Indeed, `mthcompflg` in `StkMthSecurityData` can be `Y` or `R` in  additional rows; however, all of those additional rows have `mthretflg` set to `NS` (new security) or `DE` (delists) and are hence not representing specific issues for us. Some of the above tagged returns will be removed below, for example, when appearing at the beginning of a time series. 

In [19]:
print('This step tags {:_} ({:.2f}%) rows.'.format(bl.sum(), 100 * bl.sum() / len(df)))

This step tags 38_249 (1.03%) rows.


In [20]:
df.loc[bl, 'bcktstflg'] = FLAG_PROBLEMATIC_INTERMEDIATE_RETURN   
# This step might potentially overwrite the FLAG_RETURN_BASED_ON_BA flag.  
# These flags might be overwritten below in case they correspond to a 'temporary delisting'.

## Pivoting the data

In [21]:
%%time

df = df.pivot(index='mthcaldt', columns='permno')

CPU times: user 3.79 s, sys: 599 ms, total: 4.39 s
Wall time: 4.41 s


In [22]:
# check that index is sorted
assert df['mthret'].index.is_monotonic_increasing

## Cleaning the beginning and end of each time series

####  Beginning of each time series

Let's clean a bit the *beginning* of each time series. We remove entries corresponding to returns of assets that have not yet observed a valid return. Note that the following code cells only change the beginning of each time series. The intuition behind this cleaning step is that in real-time we would only start investing in such securities as soon as they are sufficiently well traded. 

In [23]:
mask = df['mthret'].isnull() | df['bcktstflg'].gt(0) | df['mthdelflg'].isin(['G', 'M', 'V'])
mask = mask.cummin()

In [24]:
%%time
print(df.columns.levels[0])

for c in df.columns.levels[0]:
    print(c)
    print(len(df[c]))
    df[c] = df[c].mask(mask)

Index(['mthprevcap', 'mthret', 'mthdelflg', 'bcktstflg'], dtype='object')
mthprevcap
1164
mthret
1164
mthdelflg
1164
bcktstflg
1164
CPU times: user 4h 25min 59s, sys: 1h 18min 50s, total: 5h 44min 50s
Wall time: 8h 27min 21s


There are quite a few time series that correspond to assets in which we never would start investing according to this rule above. Many of these price time series have never observed trading prices (i.e. `mthprcflg` is set to `BA` throughout).

We remove them from the dataframe in the following:

In [25]:
bl = mask.iloc[-1]

In [26]:
print('There are {} ({:.2f}%) time series without a valid return after removing missing data at the beginning.'.format(
            bl.sum(), 100 * bl.mean()))

There are 4127 (15.59%) time series without a valid return after removing missing data at the beginning.


In [27]:
df = df.loc[:, df.columns.get_level_values('permno').isin(bl.index[~bl])]

#### Delisting and cleaning the end of the time series

We now remove missing returns at the end of each time series and set missing delisting returns to `MISSING_DELIST_RETURN`.  Note that a time series need not have a regular or missing delisting return; for example, if the security is still traded or fell out of the investment universe because of a change in status (e.g. change of `usincflg`).

To understand the values of the related column `mthdelflg`, see [here](https://www.crsp.org/files/appendix/FlagType_DE.html).

In [28]:
mask = df['mthret'].isnull() | df['bcktstflg'].gt(0) | df['mthdelflg'].isin(['G', 'M', 'V'])

mask = mask[::-1].cummin()[::-1]

mask = mask.mask(df['bcktstflg'].isnull()[::-1].cummin()[::-1], other=False)

df = df.drop('mthdelflg', axis=1, level=0)
df.columns = df.columns.remove_unused_levels()

In [29]:
print('There are {} ({:.2f}%) time series whose return series at the end are being modified.'.format(
    mask.any().sum(), 100 * mask.any().mean()))

There are 3974 (17.78%) time series whose return series at the end are being modified.


In [30]:
print('There are {} returns being modified.'.format(mask.sum().sum()))

There are 21960 returns being modified.


In [31]:
mask_first_return = mask & ~mask.shift(1, fill_value=False)   
# the first of the problematic returns at the end of each problematic time series

mask_others = mask & ~mask_first_return

In [32]:
df['mthret'] = df['mthret'].mask(mask_first_return, 
                                 other=df['mthret'].fillna(0).add(1).multiply(1 + MISSING_DELIST_RETURN).subtract(1))
df['bcktstflg'] = df['bcktstflg'].mask(mask_first_return, other=FLAG_DELRET_MISSING)

In [34]:
print(df.columns.levels[0])
for c in df.columns.levels[0]:
    print(c)
    df[c] = df[c].mask(mask_others)

Index(['mthprevcap', 'mthret', 'bcktstflg'], dtype='object')
mthprevcap
mthret
bcktstflg


In [35]:
# Check that each return time series has at least one value
assert df['mthret'].notnull().any().all()

## Temporary delistings

We now take care of missing returns for securities in the investment universe. Note that by the above manipulations, returns on delisting dates always exist.

In [36]:
mask = df['mthret'].isnull() & df['bcktstflg'].notnull()

We distinguish two cases: (a) the previous return exists (security was in the investment universe) and is not problematic (with corresponding flag equal to zero or `FLAG_RETURN_BASED_ON_BA`); 
(b) the previous return is problematic, doesn't exist, or the security was not in the investment universe.

In case (a), we consider this situation as a temporary delisting.  In case (b), we assume any temporary delisting effects are already taken into account by the previous returns and we remove the security from the investment universe for that month.

In [37]:
mask_tmp_delist = mask & df['bcktstflg'].shift(1).isin([0, FLAG_RETURN_BASED_ON_BA])
mask_remove = mask & ~mask_tmp_delist

In [38]:
print("""There are {} ({:.2f}%) time series that have temporary delistings (mthprevcap is given but return is 
missing and previous return was good).""".format(
      mask_tmp_delist.any().sum(), 100 * mask_tmp_delist.any().mean()))
print("In total, we have {} temporary delistings.".format(mask_tmp_delist.sum().sum()))

There are 57 (0.25%) time series that have temporary delistings (mthprevcap is given but return is 
missing and previous return was good).
In total, we have 57 temporary delistings.


In [39]:
df['mthret'] = df['mthret'].mask(mask_tmp_delist, other=TEMPORARY_DELISTING_RETURN)
df['bcktstflg'] = df['bcktstflg'].mask(mask_tmp_delist, other=FLAG_TEMPORARY_DELISTING)

In [40]:
print(df.columns.levels[0])
for c in df.columns.levels[0]:
    print(c)
    df[c] = df[c].mask(mask_remove)

Index(['mthprevcap', 'mthret', 'bcktstflg'], dtype='object')
mthprevcap
mthret
bcktstflg


## Flagging very large/small returns

In [41]:
mask = df['mthret'].gt(CUTOFF_LARGE_RETURN) | df['mthret'].lt(CUTOFF_SMALL_RETURN)

In [42]:
print('There are {} very large/small returns, which will be flagged.'.format(mask.sum().sum()))

There are 19147 very large/small returns, which will be flagged.


In [43]:
print('These very large/small returns appear in {} ({:.2f}%) time series.'.format(
            mask.any().sum(), 100 * mask.any().mean()))

These very large/small returns appear in 8467 (37.88%) time series.


In [44]:
df['bcktstflg'] = df['bcktstflg'].mask(mask, other=df['bcktstflg'].add(10))

## Storing

In [40]:
with pd.HDFStore(DATAPATH + 'CRSP_monthly_cleaned.h5') as store:
    store['df'] = df

## Some quick summary statistics:

In [ ]:
df['bcktstflg'].stack().value_counts().sort_index()

In [ ]:
df['bcktstflg'].stack().value_counts(normalize=True).sort_index()

In [ ]:
db.close()

In [ ]:
del df